In [4]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1 - Carga dos Dados
# Usando o CSV gerado no Script 01 (agora com Precipitação e Visibilidade)
df_full = pd.read_csv('../dataset/well_breathe_2025_full.csv')

# 2 - Definição do Alvo (Target)
# Definimos o que é risco: PM2.5 > 25 (Limite da OMS)
df_full['risk_label'] = (df_full['air_quality_PM2.5'] > 25).astype(int)

# 3 - Seleção de Features (As mesmas 5 do Simulador do App)
features = [
    'temperature_celsius', 
    'humidity', 
    'wind_kph', 
    'precip_mm', 
    'visibility_km'
]
X = df_full[features]
y = df_full['risk_label']

# 4 - Treino e Teste (Essencial para o relatório PISI 3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5 - Geração de Métricas para o seu Relatório Escrito
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisão do Modelo: {accuracy * 100:.2f}%")
print("\nRelatório Técnico de Classificação:")
print(classification_report(y_test, y_pred))

# 6 - Atualização do JSON (Para manter o selo de Risco IA no App)
json_path = '../../mobile-app/assets/data/well_breathe_cities.json'

with open(json_path, 'r', encoding='utf-8') as f:
    cities_data = json.load(f)

for city in cities_data:
    # Preparação dos dados para a predição automática
    input_df = pd.DataFrame([[
        city['worst_day']['temp'],
        city['worst_day']['hum'],
        city['worst_day']['wind'],
        city['worst_day'].get('precip', 0),
        city['worst_day'].get('vis', 10)
    ]], columns=features)
    
    prediction = model.predict(input_df)[0]
    city['ia_risk_prediction'] = "Elevado" if prediction == 1 else "Baixo"

# 7 - Exportação Final
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(cities_data, f, indent=2, ensure_ascii=False)

print("\nJSON atualizado com sucesso com a inteligência do modelo treinado.")

Precisão do Modelo: 73.65%

Relatório Técnico de Classificação:
              precision    recall  f1-score   support

           0       0.78      0.85      0.81      9670
           1       0.61      0.50      0.55      4550

    accuracy                           0.74     14220
   macro avg       0.69      0.67      0.68     14220
weighted avg       0.73      0.74      0.73     14220


JSON atualizado com sucesso com a inteligência do modelo treinado.
